In [1]:
# Adapted from work by @kbrose
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# The following code is adopted from Pat's Rolling Rain N-Year Threshold.pynb
# Loading in hourly rain data from CSV, parsing the timestamp, and adding it as an index so it's more useful

rain_df = pd.read_csv('data/ohare_full_precip_hourly.csv')
rain_df['datetime'] = pd.to_datetime(rain_df['datetime'])
rain_df = rain_df.set_index(pd.DatetimeIndex(rain_df['datetime']))
rain_df = rain_df['19700101':]
chi_rain_series = rain_df['HOURLYPrecip'].resample('1H').max()
print(rain_df.shape)

(468253, 5)


In [3]:
n_year_threshes = pd.read_csv('data/n_year_definitions.csv')
n_year_threshes = n_year_threshes.set_index('Duration')
n_year_threshes

,1-year,2-year,5-year,10-year,25-year,50-year,100-year
Duration,,,,,,,
10-day,4.12,4.95,6.04,6.89,8.18,9.38,11.14
5-day,3.25,3.93,4.91,5.70,6.93,8.04,9.96
72-hr,2.93,3.55,4.44,5.18,6.32,7.41,8.78
48-hr,2.70,3.30,4.09,4.81,5.88,6.84,8.16
24-hr,2.51,3.04,3.80,4.47,5.51,6.46,7.58
18-hr,2.30,2.79,3.50,4.11,5.06,5.95,6.97
12-hr,2.18,2.64,3.31,3.89,4.79,5.62,6.59
6-hr,1.88,2.28,2.85,3.35,4.13,4.85,5.68
3-hr,1.60,1.94,2.43,2.86,3.53,4.14,4.85


In [4]:
n_year_threshes.loc['5-min', '1-year']

0.29999999999999999

In [5]:
dur_str_to_hours = {
    '5-min':5/60.0,
    '10-min':10/60.0,
    '15-min':15/60.0,
    '30-min':0.5,
    '1-hr':1.0,
    '2-hr':2.0,
    '3-hr':3.0,
    '6-hr':6.0,
    '12-hr':12.0,
    '18-hr':18.0,
    '24-hr':24.0,
    '48-hr':48.0,
    '72-hr':72.0,
    '5-day':5*24.0,
    '10-day':10*24.0
}

In [6]:
# I am assuming here that a single hour cannot be part of more than one storm in the event_endtimes list.
# Therefore, I am looping through the list and throwing out any storms that include hours from heavier storms in the
# same block of time.=
def get_storms_without_overlap(event_endtimes, hours):
    times_taken = []
    ret_val = []
    for timestamp, hourly_precip in event_endtimes.iteritems():
        times_here = []
        for h in range(hours):
            times_here.append(timestamp - pd.DateOffset(hours=h))
        if not bool(set(times_here) & set(times_taken)):
            times_taken.extend(times_here)
            ret_val.append({'start': timestamp - pd.DateOffset(hours=hours), 'end': timestamp, 'inches': hourly_precip})
    return ret_val

In [8]:
import operator
n_s = [int(x.replace('-year','')) for x in list(n_year_threshes.columns.values)]
duration_strs = sorted(dur_str_to_hours.items(), key=operator.itemgetter(1), reverse=True)

2

In [15]:
def nyear_storms(n):
    n_index = n_s.index(n)
    next_n = n_s[n_index+1] if n_index != (len(n_s) - 1) else None
    all_storms = []

    for duration_tuple in duration_strs:

        duration_str = duration_tuple[0]
        low_thresh = n_year_threshes.loc[duration_str, str(n) + '-year']
        high_thresh = n_year_threshes.loc[duration_str, str(next_n) + '-year'] if next_n is not None else None
        
        duration = int(dur_str_to_hours[duration_str])
        rolling = chi_rain_series.rolling(window=int(duration), min_periods=0).sum()
        
        if high_thresh is not None:
            event_endtimes = rolling[(rolling >= low_thresh) & (rolling < high_thresh)].sort_values(ascending=False)
        else:
            event_endtimes = rolling[(rolling >= low_thresh)].sort_values(ascending=False)

        storms = get_storms_without_overlap(event_endtimes, duration)

        if len(storms) > 0:
            print("Across %s hours" % duration_str)
            for storm in storms:
                print('\t%s\t%s\t%s inches' % (storm['start'], storm['end'], storm['inches']))
            all_storms.extend(storms)

In [16]:
nyear_storms(100)

Across 10-day hours
	1987-08-13 15:00:00	1987-08-23 15:00:00	13.68 inches
	2008-09-04 12:00:00	2008-09-14 12:00:00	12.61 inches
	2003-04-30 02:00:00	2003-05-10 02:00:00	11.99 inches
	2014-01-26 17:00:00	2014-02-05 17:00:00	11.55 inches
Across 5-day hours
	1987-08-13 12:00:00	1987-08-18 12:00:00	13.12 inches
	2014-01-23 03:00:00	2014-01-28 03:00:00	10.8 inches
	2003-04-30 04:00:00	2003-05-05 04:00:00	10.44 inches
Across 72-hr hours
	2014-01-24 10:00:00	2014-01-27 10:00:00	10.8 inches
	1987-08-13 22:00:00	1987-08-16 22:00:00	10.28 inches
	2003-04-29 06:00:00	2003-05-02 06:00:00	9.3 inches
	2008-09-11 21:00:00	2008-09-14 21:00:00	9.12 inches
	2011-07-21 15:00:00	2011-07-24 15:00:00	8.85 inches
Across 48-hr hours
	2014-01-24 20:00:00	2014-01-26 20:00:00	10.8 inches
	1987-08-13 13:00:00	1987-08-15 13:00:00	9.94 inches
	2003-04-29 17:00:00	2003-05-01 17:00:00	9.27 inches
	2008-09-12 14:00:00	2008-09-14 14:00:00	8.99 inches
	2011-07-21 15:00:00	2011-07-23 15:00:00	8.65 inches
Across 24-hr hou

In [17]:
# Find all n-year storms
for n in n_s:
    print("%s-year storms" % str(n))
    nyear_storms(n)   

1-year storms
Across 10-day hours
	1980-08-15 08:00:00	1980-08-25 08:00:00	4.95 inches
	1986-09-24 23:00:00	1986-10-04 23:00:00	4.94 inches
	2001-09-18 22:00:00	2001-09-28 22:00:00	4.93 inches
	2001-10-10 02:00:00	2001-10-20 02:00:00	4.93 inches
	1990-04-30 02:00:00	1990-05-10 02:00:00	4.92 inches
	1982-12-02 06:00:00	1982-12-12 06:00:00	4.92 inches
	1975-08-20 05:00:00	1975-08-30 05:00:00	4.92 inches
	1997-02-20 01:00:00	1997-03-02 01:00:00	4.92 inches
	2014-06-20 23:00:00	2014-06-30 23:00:00	4.91 inches
	2015-06-10 19:00:00	2015-06-20 19:00:00	4.91 inches
	2013-04-17 22:00:00	2013-04-27 22:00:00	4.9 inches
	1993-06-17 19:00:00	1993-06-27 19:00:00	4.9 inches
	2004-05-12 06:00:00	2004-05-22 06:00:00	4.9 inches
	2009-06-09 09:00:00	2009-06-19 09:00:00	4.89 inches
	2010-07-14 02:00:00	2010-07-24 02:00:00	4.88 inches
	2002-08-13 20:00:00	2002-08-23 20:00:00	4.88 inches
	1987-08-16 04:00:00	1987-08-26 04:00:00	4.86 inches
	2007-08-19 10:00:00	2007-08-29 10:00:00	4.84 inches
	1989-08-02 03: